A következő jegyzetfüzetek futtatásához, ha még nem tetted meg, telepítened kell egy modellt, amely a `text-embedding-ada-002` alapmodellt használja, és a telepítés nevét az .env fájlban `AZURE_OPENAI_EMBEDDINGS_ENDPOINT`-ként kell beállítani.


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Ezután be fogjuk tölteni az Embedding Indexet egy Pandas Dataframe-be. Az Embedding Index egy `embedding_index_3m.json` nevű JSON fájlban van tárolva. Az Embedding Index tartalmazza az egyes YouTube átiratok beágyazásait egészen 2023 októberének végéig.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Most következő lépésként létrehozunk egy `get_videos` nevű függvényt, amely a lekérdezés alapján keres az Embedding Indexben. A függvény visszaadja az 5 leginkább hasonló videót a lekérdezéshez. A függvény működése a következő:

1. Először készül egy másolat az Embedding Indexről.
2. Ezután a lekérdezéshez tartozó Embedding kiszámítása történik az OpenAI Embedding API segítségével.
3. Majd létrejön egy új oszlop az Embedding Indexben, amelynek neve `similarity`. A `similarity` oszlop tartalmazza a koszinusz-hasonlóságot a lekérdezés Embeddingje és az egyes videó szegmensek Embeddingje között.
4. Ezután az Embedding Indexet a `similarity` oszlop alapján szűrjük. Csak azok a videók maradnak benne, amelyek koszinusz-hasonlósága legalább 0,75.
5. Végül az Embedding Indexet a `similarity` oszlop szerint sorba rendezzük, és visszaadjuk az 5 legjobb videót.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Ez a függvény nagyon egyszerű, csak kiírja a keresési lekérdezés eredményeit.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Először az Embedding Index egy Pandas Dataframe-be töltődik be.
2. Ezután a felhasználót megkérjük, hogy adjon meg egy lekérdezést.
3. Majd a `get_videos` függvény hívódik meg, amely az Embedding Indexben keres a lekérdezés alapján.
4. Végül a `display_results` függvény jeleníti meg az eredményeket a felhasználónak.
5. Ezután a felhasználót ismét megkérjük, hogy adjon meg egy újabb lekérdezést. Ez a folyamat addig ismétlődik, amíg a felhasználó be nem írja, hogy `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.hu.png)

A rendszer meg fogja kérni, hogy adjon meg egy lekérdezést. Írjon be egy lekérdezést, majd nyomja meg az entert. Az alkalmazás visszaad egy listát azokról a videókról, amelyek kapcsolódnak a lekérdezéshez. Az alkalmazás egy linket is ad ahhoz a videórészlethez, ahol a kérdésre adott válasz található.

Íme néhány példa lekérdezés, amit kipróbálhat:

- Mi az az Azure Machine Learning?
- Hogyan működnek a konvolúciós neurális hálózatok?
- Mi az a neurális hálózat?
- Használhatom a Jupyter Notebookokat az Azure Machine Learninggel?
- Mi az az ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**Jogi nyilatkozat**:  
Ez a dokumentum AI fordítási szolgáltatás, a [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével készült. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti, eredeti nyelvű dokumentum tekintendő hiteles forrásnak. Kritikus információk esetén javasoljuk a professzionális, emberi fordítást. Nem vállalunk felelősséget a fordítás használatából eredő félreértésekért vagy téves értelmezésekért.
